In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('ddam_project')
         .config('spark.some.config.option','some-value')
         .getOrCreate()
         )

In [1]:
# read file from hdfs and infer schema

df_cleaned = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/df_cleaned.csv", header = True, inferSchema = True)
df_cleaned.printSchema()

NameError: name 'spark' is not defined

In [3]:
df_cleaned.count()

504989

In [ ]:
df_raw = 

# VADER evaluation

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

In [5]:
vader = SentimentIntensityAnalyzer()

In [6]:
# applying vader to each review and standardize score in [0,10]
rdd_scores = (df_cleaned.select("Reviewer_Nationality", "Review", "Reviewer_Score").rdd
              #.filter(lambda row: row['Review'] is not None)
              .map(lambda x: (x["Reviewer_Nationality"], 
                              ((vader.polarity_scores(x["Review"])['compound']  ) +1)*5,
                              x["Reviewer_Score"]))
             )

#.toDF(["id", "Vader_scores", "User_scores"])
#df_scores.printSchema()

In [7]:
rdd_scores.take(10)

[('Canada', 9.1835, 8.8),
 ('United Kingdom', 8.591999999999999, 9.2),
 ('United Kingdom', 9.8475, 8.8),
 ('United Kingdom', 5.0, 9.6),
 ('United Kingdom', 5.0, 9.2),
 ('Australia', 9.643999999999998, 10.0),
 ('United Kingdom', 3.4684999999999997, 5.0),
 ('United Kingdom', 9.907, 9.2),
 ('United Kingdom', 9.5775, 9.6),
 ('Brazil', 8.7125, 9.2)]

# Evaluation

In [8]:
def errors(rdd, ix, iy):
    ''' 
    rdd= rdd composed by lists/tuples 
    ix: index of the first variable
    iy: index of the second variable
    return: dictionary with 
    {'rmse': root mean squared error
     'mae' : mean absolute error
    }
    
    '''
    # acc = ('counts', sum of abs errors', 'sum of squared errors')
    acc = (0, 0, 0)
    def mergeValue(acc, value):
        return (acc[0] + 1, acc[1] + value,  acc[2] + value**2)
    
    def mergeAccum(acc1,acc2):
        return (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2]+acc2[2])
    
    acc = (rdd.map(lambda x: ( np.abs(x[ix] - x[iy]) ))
       .aggregate(acc, mergeValue, mergeAccum)
      )
    
    return {'rmse':np.sqrt(acc[2]/acc[0]) , 'mae': acc[1]/acc[0] }
    
    
    

In [9]:
rdd_scores.take(20)

[('Canada', 9.1835, 8.8),
 ('United Kingdom', 8.591999999999999, 9.2),
 ('United Kingdom', 9.8475, 8.8),
 ('United Kingdom', 5.0, 9.6),
 ('United Kingdom', 5.0, 9.2),
 ('Australia', 9.643999999999998, 10.0),
 ('United Kingdom', 3.4684999999999997, 5.0),
 ('United Kingdom', 9.907, 9.2),
 ('United Kingdom', 9.5775, 9.6),
 ('Brazil', 8.7125, 9.2),
 ('Netherlands', 9.749, 8.3),
 ('United Kingdom', 9.338, 9.6),
 ('United Kingdom', 7.1075, 9.2),
 ('India', 5.0, 9.6),
 ('United Kingdom', 5.7655, 7.1),
 ('Italy', 3.9715, 7.1),
 ('United Kingdom', 9.1125, 9.6),
 ('Sweden', 8.5015, 8.3),
 ('Hong Kong', 8.8515, 7.5),
 ('United Kingdom', 2.213, 8.3)]

In [10]:
vader_errors = errors(rdd_scores, 1, 2)
vader_errors

{'mae': 1.6846593123810762, 'rmse': 2.3336413418498716}

# Nationality Bias

### mean bias correction
first we add the mean bias of each country to the vader score  
Correcting the score using the mean bias, the vader performance increase?


In [12]:
df_cleaned = df_cleaned.select( "Review", "Reviewer_Nationality", "Reviewer_Score")

In [13]:
df_bias_mean = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/df_bias_mean.csv", header = True, inferSchema = True)
df_bias_mean = df_bias_mean.select("Reviewer_Nationality", "Bias")

In [14]:
df_joined = df_cleaned.join(df_bias_mean, 'Reviewer_Nationality', 'left')

In [15]:
print(df_joined.count())
df_joined.show(10)

504989
+--------------------+--------------------+--------------+--------------------+
|Reviewer_Nationality|              Review|Reviewer_Score|                Bias|
+--------------------+--------------------+--------------+--------------------+
|              Canada|staff in restaura...|           8.8|  0.1549723388282711|
|      United Kingdom|. overall hotel w...|           9.2| 0.08905211107880007|
|      United Kingdom|no bad experience...|           8.8| 0.08905211107880007|
|      United Kingdom|room is small any...|           9.6| 0.08905211107880007|
|      United Kingdom|my son and his fa...|           9.2| 0.08905211107880007|
|           Australia|nothing to improv...|          10.0| 0.19915391522263093|
|      United Kingdom|my and my wife ca...|           5.0| 0.08905211107880007|
|      United Kingdom|expensive but thi...|           9.2| 0.08905211107880007|
|      United Kingdom|nothing not to li...|           9.6| 0.08905211107880007|
|              Brazil|. rooms are

In [16]:
try:
    df_joined.createTempView('bias_mean')
except:
    spark.catalog.dropTempView('bias_mean')
    df_joined.createTempView('bias_mean')

In [17]:
spark.sql("SELECT Reviewer_Nationality, count(*) FROM bias_mean WHERE Bias is null GROUP BY Reviewer_Nationality ").show()

+--------------------+--------+
|Reviewer_Nationality|count(1)|
+--------------------+--------+
|Turks Caicos Islands|      14|
|            Paraguay|      28|
|            Anguilla|       1|
|               Yemen|      16|
|          St Maarten|      11|
|             Senegal|      22|
|            Kiribati|       2|
|              Guyana|       5|
|             Eritrea|       2|
|            Djibouti|       2|
|                Fiji|      11|
|              Malawi|      10|
|Northern Mariana ...|       1|
|             Comoros|       1|
|            Cambodia|      32|
|         Afghanistan|       7|
|              Crimea|       6|
|            Maldives|      47|
|              Rwanda|      12|
|         Ivory Coast|      19|
+--------------------+--------+
only showing top 20 rows



In [18]:
def vader_bias(x):
  
    if x["Bias"] == None:
        score = (((vader.polarity_scores(x["Review"])['compound']  ) +1)*5 ) + 0
    else:
        score = (((vader.polarity_scores(x["Review"])['compound']  ) +1)*5 ) + x["Bias"]
        
    # handling scores 'overflow'
    if score > 10:
        score = 10
    elif score < 0:
        score = 0
        
    return (x['Reviewer_Score'], score)

In [19]:
rdd_bias_mean = (df_joined.rdd
              #.filter(lambda row: row['Review'] is not None) 
              .map(vader_bias)
             )
rdd_bias_mean.take(20)

[(8.8, 9.338472338828272),
 (9.2, 8.681052111078799),
 (8.8, 9.9365521110788),
 (9.6, 5.0890521110788),
 (9.2, 5.0890521110788),
 (10.0, 9.84315391522263),
 (5.0, 3.5575521110787998),
 (9.2, 9.9960521110788),
 (9.6, 9.6665521110788),
 (9.2, 8.752844487213734),
 (8.3, 9.478764599475605),
 (9.6, 9.4270521110788),
 (9.2, 7.1965521110788),
 (9.6, 4.487335634316863),
 (7.1, 5.8545521110788),
 (7.1, 3.6853268377741637),
 (9.6, 9.2015521110788),
 (8.3, 8.277453199096339),
 (7.5, 8.482835961193082),
 (8.3, 2.3020521110788)]

In [20]:
vader_errors_bias_mean = errors(rdd_bias_mean, 0, 1)

In [21]:
vader_errors_bias_mean

{'mae': 1.6907125757447694, 'rmse': 2.3470459578524867}

In [22]:
vader_errors

{'mae': 1.6846593123810762, 'rmse': 2.3336413418498716}

The performance are not improved using the mean bias correction, let's try with the median

### Median bias correction

In [11]:
df_bias_median = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/df_bias_median.csv", header = True, inferSchema = True)
df_bias_median = df_bias_median.select("Reviewer_Nationality", "Bias_median", "N").orderBy('Bias_median')
df_bias_median.show(100)

+--------------------+-------------------+------+
|Reviewer_Nationality|        Bias_median|     N|
+--------------------+-------------------+------+
|               Macau|-0.9000000000000004|   105|
|          Azerbaijan|-0.9000000000000004|   262|
|          Bangladesh|-0.9000000000000004|   140|
|            Pakistan|-0.9000000000000004|   874|
|                Iran|-0.9000000000000004|  1033|
|               Ghana|               -0.5|   137|
|           Hong Kong|               -0.5|  2967|
|           Singapore|               -0.5|  3047|
|                Oman|               -0.5|  1295|
|                null|               -0.5|   511|
|           Indonesia|               -0.5|  1493|
|        Saudi Arabia|               -0.5|  8413|
|         Switzerland|               -0.5|  8555|
|             Belgium|               -0.5|  5918|
|United Arab Emirates|               -0.5|  9779|
|               Qatar|               -0.5|  2583|
|               Kenya|               -0.5|   258|


In [12]:
df_bias = df_bias_mean.join(df_bias_median, 'Reviewer_Nationality', 'inner').orderBy('Bias_median', ascending = False)

NameError: name 'df_bias_mean' is not defined

In [ ]:
# looking at Bias (mean) vs Bias median
df_bias.show(200)

In [ ]:
df_bias.filter("Reviewer_Nationality == 'Italy'" ).collect()[0]['Bias_median']

In [ ]:
df_joined_median = df_cleaned.join(df_bias_median, 'Reviewer_Nationality', 'left')

In [ ]:
try:
    df_joined_median.createTempView('bias_median')
except:
    spark.catalog.dropTempView('bias_median')
    df_joined_median.createTempView('bias_median')

In [ ]:
def vader_bias_median(x):
  
    if x["Bias_median"] == None:
        score = (((vader.polarity_scores(x["Review"])['compound']  ) +1)*5 ) + 0+1
    else:
        score = (((vader.polarity_scores(x["Review"])['compound']  ) +1)*5 ) + x["Bias_median"]+1
        
    # handling scores 'overflow'
    if score > 10:
        score = 10
    elif score < 0:
        score = 0
        
    return (x['Reviewer_Nationality'], x['Reviewer_Score'], score)

In [ ]:
rdd_bias_median = (df_joined_median.rdd
              #.filter(lambda row: row['Review'] is not None) 
              .map(vader_bias_median)
             )
rdd_bias_median.take(20)

In [73]:
vader_errors_bias_median = errors(rdd_bias_median, 1, 2)

In [74]:
vader_errors_bias_median

{'mae': 1.4522040371175613, 'rmse': 2.0167173720720903}

In [31]:
vader_errors_bias_median

{'mae': 1.6590399068098154, 'rmse': 2.307100807665857}

In [32]:
vader_errors_bias_mean

{'mae': 1.6907125757447694, 'rmse': 2.3470459578524867}

In [33]:
vader_errors

{'mae': 1.6846593123810762, 'rmse': 2.3336413418498716}

Le performance globali sono praticamente le stesse, solo impercettibilmente migliorate applicando il bias mediano.  

Globalmente non si hanno miglioramenti significativi.  

Ma, (ovviamente) le performance sui singoli paesi migliorano?





## Valutazione bias su singole nazioni/hotels

In [75]:
rdd_bias_median.filter(lambda x: x[0]== 'United States of America').take(5)

[('United States of America', 10.0, 6.399999999999999),
 ('United States of America', 8.8, 10),
 ('United States of America', 7.5, 2.8524999999999983),
 ('United States of America', 8.3, 10),
 ('United States of America', 6.3, 2.8974999999999986)]

In [76]:
errors(rdd_bias_median.filter(lambda x: x[0]=="United States of America"), 1,2 )

{'mae': 1.2088762719637625, 'rmse': 1.814910144740996}

In [41]:
errors(rdd_scores.filter(lambda x: x[0]=="United States of America"), 1,2 )

{'mae': 1.529733969673516, 'rmse': 2.2167606638808044}

In [60]:
errors(rdd_bias_median.filter(lambda x: x[0]=="Iran"), 1,2 )

{'mae': 2.4091902226524686, 'rmse': 3.08254584146196}

In [61]:
rdd_bias_median.filter(lambda x: x[0]=="Iran").take(5)

[('Iran', 6.7, 4.1),
 ('Iran', 8.8, 6.9594999999999985),
 ('Iran', 9.6, 8.803999999999998),
 ('Iran', 7.0, 5.320499999999999),
 ('Iran', 10.0, 4.1)]

In [62]:
errors(rdd_scores.filter(lambda x: x[0]=="Iran"), 1,2 )

{'mae': 1.9500396902226527, 'rmse': 2.581911881942934}

In [ ]:
from pyspark.mllib.stat import Statistics

In [63]:
summary = Statistics.colStats(rdd_scores
                              .filter(lambda x: x['Reviewer_Nationality'=='Italy'])
                              .map(lambda x: (x[1], x[2])))

In [64]:
summary.mean()

array([7.52036318, 8.3932562 ])

Plots

In [ ]:
###########################################################
import matplotlib.pyplot as plt
SMALL_SIZE = 15
MEDIUM_SIZE = 20
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y label
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

###########################################################


# Train/Test